### Import required packages

In [13]:
from osgeo import gdal, osr, ogr
import matplotlib.pyplot as plt
from osgeo.gdalconst import *
import numpy as np
import os
import osmnx as ox
import rasterio
from rasterio.merge import merge

### Set Constants
Input directory should contain .tif files in the base directory (program does not walk subdirectories). To mosaic an image, place mosaic inputs into a folder within the input directory. Separate mosaic sets should be in separate folders. The folder name will be used for mosaic output. 

The intermediate directory (INTER_DIR) contains .tif outputs for files with a cropped extent and resampled resolution.

Target extent and resolution is in EPSG:4326.

In [41]:
# CONSTANTS 
INPUT_DIR = "input/"
INTER_DIR = "temp/"
OUTPUT_DIR = "output/"
# [ulx, uly, lrx, lry]
# = [x_min, y_max, x_max, y_min]
TARGET_EXT = [47.42053671268654, -18.809913565707074, 47.62104068410202, -19.01041753712255]
TARGET_RES = [0.00026949458523585647, 0.00026949458523585647]

### Mosaic Images
Merges raster images within folders in INPUT_DIR. Folder names will be used for output filename. 

In [15]:
def mosaicImages(directory):
    pathList = []
    rasterioObjList = []
    outName = ""
    for dirs, subdirs, files in os.walk(directory):
        outName = os.path.basename(dirs)
        for f in files:
            path = dirs + os.sep + f
            if path.endswith(".tif"):
                src = rasterio.open(path)
                pathList.append(path)
                rasterioObjList.append(src)
             
    # Create mosaic image        
    mosaic, out_t = merge(rasterioObjList)
    out_meta = rasterioObjList[0].meta.copy()
    out_meta.update({
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_t
    })
    
    # Save mosaic image
    with rasterio.open(INPUT_DIR + outName + ".tif", "w", **out_meta) as dest:
        dest.write(mosaic)
        
    # Set mosaic image band names 
    inDs = gdal.Open(pathList[0])
    mosaicDs = gdal.Open(INPUT_DIR + outName + ".tif")
    for i in range(inDs.RasterCount):
        mosaicDs.GetRasterBand(i+1).SetDescription(inDs.GetRasterBand(i+1).GetDescription())
        
    mosaicDs = None

### Get Extent and Resolution
If you wish to copy the extent and resolution of a single input file for the final stacked output, identify values using the following two functions.

In [16]:
def getExtent(path):
    raster_file = gdal.Open(path,0)
    geoTransform = raster_file.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1]*raster_file.RasterXSize
    miny = maxy + geoTransform[5]*raster_file.RasterYSize
#     return [minx,miny,maxx,maxy]
    return [minx,maxy,maxx,miny]

In [17]:
def getResolution(path):
    reference = gdal.Open(path, 0)
    rt = reference.GetGeoTransform()
    xr = rt[1]
    yr = -rt[5]
    return [xr, yr]

### Check Extent Cover 
Confirm that raster covers the desired output extent

In [42]:
def checkExtent(ds):
    # Source: https://gis.stackexchange.com/questions/57834/how-to-get-raster-corner-coordinates-using-python-gdal-bindings
    
    ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()
    lrx = ulx + (ds.RasterXSize * xres)
    lry = uly + (ds.RasterYSize * yres)
    
    source = osr.SpatialReference()
    source.ImportFromWkt(ds.GetProjection())

    # Target projection
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)

    # Transformation function
    transform = osr.CoordinateTransformation(source, target)

    # Transform points
    ulx, uly, ret = transform.TransformPoint(ulx, uly)
    lrx, lry, ret = transform.TransformPoint(lrx, lry)
    
    if( ulx <= TARGET_EXT[0] and uly >= TARGET_EXT[1] and lrx >= TARGET_EXT[2] and lry <= TARGET_EXT[3]):
        return True
    else:
        return False
#     print(ulx, uly, lrx, lry)
    
    
    

### Generate Intermediate Images
Edits all input rasters and mosaics to desired extent and resolution and saves products to INTER_DIR. If needed, change resampling algorithm from "gdal.GRIORA_Bilinear" to any desired value. 

In [49]:
# Count total number of layers for future stacking 
numLayers = 0

# Check if intermediate path exists
if not os.path.exists(INTER_DIR):
    os.mkdir(INTER_DIR)
    
# Iterate through input directory and find .tif files
input_tifs = []
for subdir, dirs, files in os.walk(INPUT_DIR):
    for d in dirs:
        mosaicImages(INPUT_DIR + d)
for subdir, dirs, files in os.walk(INPUT_DIR):
    for f in files:
        path = subdir + os.sep + f
        if path.endswith(".tif") or path.endswith(".JP2"):
            input_tifs.append(path)
    break;
    
# Resize, resample and save to intermediate dir
for tif in input_tifs:
    inputDs = gdal.Open(tif)
    if(checkExtent(inputDs)):
        numLayers += inputDs.RasterCount
        ds = gdal.Warp(str(INTER_DIR + os.path.basename(tif)), inputDs,
                       xRes=TARGET_RES[0], yRes=TARGET_RES[1],
                       outputBounds=TARGET_EXT,
                       resampleAlg= gdal.GRIORA_Bilinear,
                       dstSRS='EPSG:4326',
        #                format='MEM'
                       format='GTiff'
                      )
        ds = None
    else:
        print(tif + " does not contain specified target extent. Excluding file from output.");


input//highres.JP2 does not contain specified target extent. Excluding file from output.


### Get Intermediate Product List
Iterates through intermediate product folder. Also identifies constants for final stacked raster output.

In [50]:
# Iterate through intermediate product directory
tifs = []
for subdir, dirs, files in os.walk(INTER_DIR):
    for f in files:
        path = subdir + os.sep + f
        if path.endswith(".tif"):
            tifs.append(path)
            
# Get shape for output file from sample intermediate prod
inputDs = gdal.Open(tifs[0])
geo_transform = inputDs.GetGeoTransform()
out_shape = inputDs.GetRasterBand(1).ReadAsArray().shape
inputDs = None 

### Build and Export Stacked Raster
Exports multilayer.tif to OUTPUT_DIR. Saves band names from original files, if they exist. 

In [51]:
driver = gdal.GetDriverByName("Gtiff")
outDataset = driver.Create(OUTPUT_DIR + "multilayer.tif", out_shape[1], out_shape[0], numLayers, gdal.GDT_Float32)
outDataset.SetGeoTransform(geo_transform)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)
outDataset.SetProjection(srs.ExportToWkt())

currLayer = 1
for tif in tifs:
    tempFile = gdal.Open(tif)
    for ind in range(tempFile.RasterCount):
        fileLayerArr = tempFile.GetRasterBand(ind+1).ReadAsArray()
        outDataset.GetRasterBand(currLayer).WriteArray(fileLayerArr)
        outDataset.GetRasterBand(currLayer).SetDescription(tempFile.GetRasterBand(ind+1).GetDescription())
        currLayer += 1
outDataset = None